<a href="https://colab.research.google.com/github/MJEFFTKM/The_cheapest_taxi_bot/blob/main/CheapTaxi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-telegram-bot

     |████████████████████████████████| 430kB 5.8MB/s 
     |████████████████████████████████| 61kB 5.7MB/s 
     |████████████████████████████████| 2.6MB 8.3MB/s 


In [ ]:
!pip install python—telegram-bot
!pip3 install PyTelegramBotAPI==3.7.4
!pip install uber-rides

ERROR: Invalid requirement: 'python—telegram-bot'


In [ ]:
from uber_rides.auth import AuthorizationCodeGrant
auth_flow = AuthorizationCodeGrant(
    'Nvcji2FLs9TV1SCr7yo4VpcpH4QIIw7B',
    <SCOPES>,
    'ekYeaTTQlp1sd9gA3nkRISb5kxCqsBTO9p2QVtii',
    <REDIRECT_URI>
)
auth_url = auth_flow.get_authorization_url()
session = auth_flow.get_session(redirect_url)

TypeError: ignored

In [ ]:
client = UberRidesClient(session)

response = client.get_price_estimates(
    start_latitude=37.770,
    start_longitude=-122.411,
    end_latitude=37.791,
    end_longitude=-122.405,
    seat_count=2
)

estimate = response.json.get('prices')

TypeError: ignored

In [ ]:
import bs4
import selenium


ModuleNotFoundError: ignored

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#Подлкючаем библиотеку requests
import requests
import logging
import bs4
#Подключаем нужные для бота модули из библиотеки telegram.ext
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, ConversationHandler, CallbackContext
from telegram import InlineKeyboardButton
from telegram import InlineKeyboardMarkup
from telegram import ReplyKeyboardMarkup, ReplyKeyboardRemove, Update
from telegram import Bot

CLID = "ak1507"
APIKEY = "03a97db47ad24327b7c9f5e5551658e7"

location_start = ''
location_end = ''

class_of_taxi = ''

start_lon = ''
start_lat = ''

end_lon = ''
end_lat = ''

START, CHECK_START, END, CHECK_END, CLASS  = range(5)


def yandex_price():
    global CLID, APIKEY, location_start, location_end, class_of_taxi, start_lon, start_lat, end_lon, end_lat
    level_class = ""
    print(class_of_taxi)
    if class_of_taxi == 'Эконом':
        level_class = 'econom'
    elif class_of_taxi == 'Комфорт':
        level_class = 'business'
    else:
        level_class = 'comfortplus'
    PARAMS = {
    "clid": CLID,
    "apikey": APIKEY,
    "rll": start_lon + "," + start_lat + "~" + end_lon + "," + end_lat,
    "class": level_class,
    }
    print(type(start_lon), type(start_lat), type(end_lon), type(end_lat))
    r = requests.get(url="https://taxi-routeinfo.taxi.yandex.net/taxi_info?", params=PARAMS)
    r = r.json()
    return r['options'][0]['price_text'], str(r['options'][0]['waiting_time'] // 60)[:-2]

def uber_price():
    global CLID, APIKEY, location_start, location_end, class_of_taxi, start_lon, start_lat, end_lon, end_lat
    level_class = ""
    print(class_of_taxi)
    if class_of_taxi == 'Эконом':
        level_class = 'econom'
    elif class_of_taxi == 'Комфорт':
        level_class = 'business'
    else:
        level_class = 'comfortplus'

def get_coords_from_address(adress):
    PARAMS = {
    "apikey": "bb01a0bf-02f4-459a-9764-1b854cbce432",
    "format": "json",
    "lang": "ru_RU",
    "kind": "house",
    "geocode": adress
    }
    
    try:

        r = requests.get(url="https://geocode-maps.yandex.ru/1.x/", params=PARAMS)
        json_data = r.json()
        lon, lat = json_data['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'].split()
        address_str = json_data['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['metaDataProperty']['GeocoderMetaData']['AddressDetails']['Country']['AddressLine']
        return address_str, lon, lat
    #долгота = 37.47174
    #широта = 55.652999
    except Exception as e:
        #единственное что тут изменилось, так это сообщение об ошибке.
        return ""

def get_address_from_coords(coords):
    PARAMS = {
        "apikey": "bb01a0bf-02f4-459a-9764-1b854cbce432",
        "format": "json",
        "lang": "ru_RU",
        "kind": "house",
        "geocode": coords
    }

    try:
        r = requests.get(url="https://geocode-maps.yandex.ru/1.x/", params=PARAMS)
        json_data = r.json()
        address_str = json_data["response"]["GeoObjectCollection"]["featureMember"][0]["GeoObject"]["metaDataProperty"][
            "GeocoderMetaData"]["AddressDetails"]["Country"]["AddressLine"]
        return address_str

    except Exception as e:
        #единственное что тут изменилось, так это сообщение об ошибке.
        return ""
        

#Эта функция будет использоваться когда человек первый нажал в боте START
def start(update, context):
    global location_start, location_end, class_of_taxi, start_lon, start_lat, end_lon, end_lat
    #эта строка отправляет сообщение пользователю с просьбой послать локацию или координаты
    update.message.reply_text("🚕Здравствуйте, я бот, который ищет самое дешевое такси откуда вам хочется, и куда вам нужно:)")
    update.message.reply_text("📍Прикрепите геолокацию (может быть неточной) или напишите адрес начала маршрута.📍")
    return START

#Эта функция будет использоваться, если пользователь послал в бота любой текст.
#Мы ожидаем координаты, но если прийдет что-то другое не страшно, ведь мы описали в функции получения адреса возвращение ошибки в случае чего.  

def class_taxi(update, context):
    global location_start, location_end, class_of_taxi, start_lon, start_lat, end_lon, end_lat
    class_of_taxi = update.message.text
    if class_of_taxi != 'Эконом' and  class_of_taxi != 'Комфорт' and class_of_taxi != 'Комфорт+':
        update.message.reply_text("К сожалению, такого класса нет;(")
        update.message.reply_text("Выберете еще раз класс на котором хотите поехать", reply_markup=ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True))
        return CLASS
    else:
        price, time = yandex_price()
        update.message.reply_text("Яндекс.GO цена: " + price + " минут ждать: " + time, reply_markup=ReplyKeyboardRemove())
        return ConversationHandler.END

def check_start(update: Updater, context: CallbackContext):
    global location_start, location_end, class_of_taxi, start_lon, start_lat, end_lon, end_lat
    ans = update.message.text
    if ans == "Закончить":
        update.message.reply_text("Надеюсь, еще увидимся😉. Когда я вам понадоблюсь, просто наберите /start", reply_markup=ReplyKeyboardRemove())
        return ConversationHandler.END
    elif ans == "Исправить":
        update.message.reply_text("📍Прикрепите геолокацию или напишите адрес начала маршрута еще раз.📍", reply_markup=ReplyKeyboardRemove())
        return START
    else:
        update.message.reply_text("Адрес начала пути: " + location_start, reply_markup=ReplyKeyboardRemove())
        update.message.reply_text("📍Прикрепите геолокацию или напишите адрес конца маршрута.📍")
        return END

def check_end(update: Updater, context: CallbackContext):
    global location_start, location_end, class_of_taxi, start_lon, start_lat, end_lon, end_lat
    ans = update.message.text
    if ans == "Закончить":
        update.message.reply_text("Надеюсь, еще увидимся😉. Когда я вам понадоблюсь, просто наберите /start", reply_markup=ReplyKeyboardRemove())
        return ConversationHandler.END
    elif ans == "Исправить":
        update.message.reply_text("📍Прикрепите геолокацию или напишите адрес конца маршрута еще раз.📍", reply_markup=ReplyKeyboardRemove())
        return END
    else:
        update.message.reply_text("Адрес начала пути: " + location_start, reply_markup=ReplyKeyboardRemove())
        update.message.reply_text("Адрес конца пути: " + location_end)
        #update.message.reply_text("Координаты начала пути: " + start_lon + " " + start_lat)
        #update.message.reply_text("Координаты конца пути: "  + end_lon + " " + end_lat)
        reply_keyboard = [['Эконом', 'Комфорт', 'Комфорт+']]
        update.message.reply_text("Выберете класс на котором хотите поехать", reply_markup=ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True))
        return CLASS


def start_loc(update: Updater, context: CallbackContext):
    global location_start, location_end, class_of_taxi, start_lon, start_lat, end_lon, end_lat
    reply_keyboard = [['Все правильно', 'Исправить', 'Закончить']]
    #получаем текст от пользователя
    address = update.message.text
    #отправляем текст в нашу функцио получения адреса из координат
    location_start, start_lon, start_lat = get_coords_from_address(address)
    #вовщращаем результат пользователю в боте
    if location_start == "":
        update.message.reply_text("К сожалению, я не смог найти такого адреса😪")
        update.message.reply_text("📍Пришлите еще раз адрес или геопозицию📍")
        return START
    else:
        update.message.reply_text(location_start)
        update.message.reply_text("Проверьте, правильно ли указал адрес?", reply_markup=ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True))
        return CHECK_START



def end_loc(update: Updater, context: CallbackContext):
    global location_start, location_end, class_of_taxi, start_lon, start_lat, end_lon, end_lat
    reply_keyboard = [['Все правильно', 'Исправить', 'Закончить']]
    #получаем текст от пользователя
    address = update.message.text
    #отправляем текст в нашу функцио получения адреса из координат
    location_end, end_lon, end_lat = get_coords_from_address(address)
    #вовщращаем результат пользователю в боте
    if location_end == "":
        update.message.reply_text("К сожалению, я не смог найти такого адреса😪")
        update.message.reply_text("📍Пришлите еще раз адрес или геопозицию📍")
        return END
    else:
        update.message.reply_text(location_end)
        update.message.reply_text("Проверьте, правильно ли указал адрес?", reply_markup=ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True))
        return CHECK_END

#Эта функция будет использоваться, если пользователь послал локацию.
def start_location(update, context):
    reply_keyboard = [['Все правильно', 'Исправить', 'Закончить']]
    global location_start, location_end, class_of_taxi, start_lon, start_lat, end_lon, end_lat
    #получаем обьект сообщения (локации)
    message = update.message
    #вытаскиваем из него долготу и ширину
    start_lon = str(message.location.longitude)
    start_lat = str(message.location.latitude)
    current_position = (start_lon, start_lat)
    #создаем строку в виде ДОЛГОТА,ШИРИНА
    coords = f"{current_position[0]},{current_position[1]}"
    #отправляем координаты в нашу функцию получения адреса
    location_start = get_address_from_coords(coords)
    if location_start == "":
        update.message.reply_text("К сожалению, я не смог найти такого адреса😪")
        update.message.reply_text("📍Пришлите еще раз адрес или геопозицию📍")
        return START
    else:
        update.message.reply_text(location_start)
        update.message.reply_text("Проверьте, правильно ли указал адрес?", reply_markup=ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True))
        return CHECK_START

def end_location(update, context):
    global location_start, location_end, class_of_taxi, start_lon, start_lat, end_lon, end_lat
    #получаем обьект сообщения (локации)
    reply_keyboard = [['Все правильно', 'Исправить', 'Закончить']]
    message = update.message
    #вытаскиваем из него долготу и ширину
    end_lon = str(message.location.longitude)
    end_lat = str(message.location.latitude)
    current_position = (end_lon, end_lat)
    #создаем строку в виде ДОЛГОТА,ШИРИНА
    coords = f"{current_position[0]},{current_position[1]}"
    #отправляем координаты в нашу функцию получения адреса
    location_end = get_address_from_coords(coords)
    #вовщращаем результат пользователю в боте
    if location_end == "":
        update.message.reply_text("К сожалению, я не смог найти такого адреса😪")
        update.message.reply_text("📍Пришлите еще раз адрес или геопозицию📍")
        return END
    else:
        update.message.reply_text(location_end)
        update.message.reply_text("Проверьте, правильно ли указал адрес?", reply_markup=ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True))
        return CHECK_END
        

    return END

def cancel(update, context):
    global location_start, location_end, class_of_taxi, start_lon, start_lat, end_lon, end_lat
    user = update.message.from_user
    logger.info("User %s canceled the conversation.", user.first_name)
    update.message.reply_text(
        'Bye! I hope we can talk again some day.')

    return ConversationHandler.END


#Это основная функция, где запускается наш бот
def main():
    #создаем бота и указываем его токен
    #создаем регистратор событий, который будет понимать, что сделал пользователь и на какую функцию надо переключиться.
    updater = Updater("1488173907:AAFf9wCXN9XsWW6NvL7K5RrO5w_yiCWGEvo", use_context=True)
    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],
        states={
            START: [MessageHandler(Filters.text, start_loc), MessageHandler(Filters.location, start_location)],
            CHECK_START: [MessageHandler(Filters.text, check_start)],
            END: [MessageHandler(Filters.text, end_loc), MessageHandler(Filters.location, end_location)],
            CHECK_END: [MessageHandler(Filters.text, check_end)],
            CLASS: [MessageHandler(Filters.text, class_taxi)]
            }, fallbacks=[CommandHandler('csdvjnsdvjhvjnvdsnksc', cancel)],
    )
    #fallbacks=[CommandHandler('cancel', cancel)]
    dispatcher.add_handler(conv_handler)
    updater.start_polling()
    updater.idle()


if __name__ == '__main__':
    #запускаем функцию def main
    main()

In [ ]:
def check(update: Updater, context: CallbackContext):

    ans = update.message.text
    print(ans)
    if ans == 'Закончить':
        return -1
    elif ans == 'Исправить':
        return 0
    else:
        return 1


def start_loc(update: Updater, context: CallbackContext):
    reply_keyboard = [['Все правильно', 'Исправить', 'Закончить']]
    #получаем текст от пользователя
    coords = update.message.text
    #отправляем текст в нашу функцио получения адреса из координат
    location_start = get_address_from_coords(coords)
    #вовщращаем результат пользователю в боте
    if location_start == "":
          update.message.reply_text("😪К сожалению, я не смог найти такого адреса, введите еще раз🔄")
          start_loc(Updater, CallbackContext)
    else:
        update.message.reply_text(location_start)
        update.message.reply_text("Проверьте, правильно ли я указал адрес?", reply_markup=ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True))
        ans = update.last
        if ans == "Все правильно":
            update.message.reply_text("📍Прикрепите геолокацию или напишите адрес конца маршрута.📍")
            return END
        elif ans == "Исправить":
            update.message.reply_text("📍Прикрепите геолокацию или напишите адрес начала маршрута еще раз.📍")
            start_location(Updater, CallbackContext)
        else:
            return ConversationHandler.END

1) сделать изначальные координаты (долгота широта) начал и конца
2) добавить класс, кол-во пассажиров, ребенок (модет быть яндекс плюс)
3) написать парсеры


In [ ]:
!pip install pprintpp

In [ ]:
#Яндек.Go на вход получает долготу и широту точек
#классы = Эконом, Комфорт, Комфорт+, Бизнес, Минивен.
#можно добавить дополнительные опции: 
#yellowcarnumber — машина с желтыми номерами.
#childchair — наличие детского кресла в машине.
#bicycle — перевозка велосипеда.
#conditioner — кондиционер в машине.
#animaltransport — перевозка животных.
#universal — машина-универсал.
#check — необходима квитанция об оплате.
#ski — перевозка лыж или сноуборда.
#waiting_in_transit — ожидание в пути.
#meeting_arriving — встреча с табличкой.
#luggage — платная перевозка багажа.

import requests
import pprintpp




CLID = "ak1507"
APIKEY = "03a97db47ad24327b7c9f5e5551658e7"
r = requests.get('https://taxi-routeinfo.taxi.yandex.net/taxi_info?clid=ak1507&apikey=03a97db47ad24327b7c9f5e5551658e7&rll=37.470941,55.653356~37.453513,55.664658&class=minivan')
r = r.json()
print(r['options'][0]['price_text'], r['time_text'])


~373 руб. 7 мин


In [ ]:
import pprintpp

PARAMS = {
    "apikey": "bb01a0bf-02f4-459a-9764-1b854cbce432",
    "format": "json",
    "lang": "ru_RU",
    "kind": "house",
    "geocode": 
}

r = requests.get(url="https://geocode-maps.yandex.ru/1.x/", params=PARAMS)
json_data = r.json()
pprintpp.pprint(json_data['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'])
pprintpp.pprint(json_data['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['metaDataProperty']['GeocoderMetaData']['AddressDetails']['Country']['AddressLine'])

долгота = 37.47174
широта = 55.652999

'37.47174 55.652999'
'Россия, Москва, проспект Вернадского, 94к4'


In [ ]:
a = 1
print(type(a))

<class 'int'>


In [ ]:
!pip install selenium

     |████████████████████████████████| 911kB 4.3MB/s 


In [ ]:
import selenium

